# Carregar dados

In [1]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet("./output/silver/rais_TODOS_ANOS_RAIS_VINC_PUB.parquet.gzip")


qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 384175



,ano,id_uf_municipio,valor_remuneracao_media,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio
0,2019,500240,0.99,17,317210,1,2,MS,Caarapo
1,2019,500570,0.00,36,212420,1,2,MS,Navirai


In [2]:
df_raca_cor = pd.read_csv("./files/dic_raca_cor.csv")
df_sexo = pd.read_csv("./files/dic_sexo.csv")
df_ordered_sinonimo = pd.read_csv("./files/ordered_sinonimo.csv")

In [3]:
# change column ocupacao_id to int
df["ocupacao_id"] = df["ocupacao_id"].astype(int)

# Juntando tabelas

In [4]:
# merge df and df_sexo by sexo_id
df = pd.merge(df, df_raca_cor, left_on="raca_cor_id",
              right_on="chave", how="left")
df = pd.merge(df, df_sexo, left_on="sexo_id", right_on="chave",
              how="left")
df = pd.merge(df, df_ordered_sinonimo, left_on="ocupacao_id",
              right_on="cbo_2002", how="left")
df.head(2)

,ano,id_uf_municipio,valor_remuneracao_media,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio,chave_x,valor_x,chave_y,valor_y,cbo_2002,sinonimo
0,2019,500240,0.99,17,317210,1,2,MS,Caarapo,2.0,Branca,1,Masculino,317210.0,Analista de service desk
1,2019,500570,0.00,36,212420,1,2,MS,Navirai,2.0,Branca,1,Masculino,212420.0,Analista de suporte


# Gerando ID para Sigla_UF

In [5]:
# Create id for sigla_uf
df["sigla_uf_id"] = df["sigla_uf"].astype("category").cat.codes
df.head(2)


,ano,id_uf_municipio,valor_remuneracao_media,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio,chave_x,valor_x,chave_y,valor_y,cbo_2002,sinonimo,sigla_uf_id
0,2019,500240,0.99,17,317210,1,2,MS,Caarapo,2.0,Branca,1,Masculino,317210.0,Analista de service desk,11
1,2019,500570,0.00,36,212420,1,2,MS,Navirai,2.0,Branca,1,Masculino,212420.0,Analista de suporte,11


# Selecionando colunas necessárias

In [6]:
df = df[["ano", "sigla_uf_id", "sigla_uf", "valor_remuneracao_media", "idade",
         "ocupacao_id", "sinonimo", "sexo_id", "valor_y", "raca_cor_id", "valor_x"]]

df = df.rename(columns={
    "sinonimo": "ocupacao",
    "valor_y": "sexo", 
    "valor_x": "raca_cor"
    })

df.head(2)

,ano,sigla_uf_id,sigla_uf,valor_remuneracao_media,idade,ocupacao_id,ocupacao,sexo_id,sexo,raca_cor_id,raca_cor
0,2019,11,MS,0.99,17,317210,Analista de service desk,1,Masculino,2,Branca
1,2019,11,MS,0.00,36,212420,Analista de suporte,1,Masculino,2,Branca


# Salvando arquivo

In [7]:
df.to_parquet("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip", index=False, compression="gzip")